In [1]:
%load_ext autoreload
%autoreload 2

from MyClassifier_13 import MyClassifier
import numpy as np
import requests, gzip, os, hashlib

np.random.seed(1) # for reproducibility

In [2]:
### driver code
use_mnist = False # Set to True to use MNIST, False to use synthetic data (smaller and faster)

if use_mnist:
    # fetching the MNIST data

    ### change this line of code to put the data where you want it
    path = "./testing"
    os.makedirs(path, exist_ok=True)
    ###

    def fetch(url):
        fp = os.path.join(path, hashlib.md5(url.encode('utf-8')).hexdigest())
        if os.path.isfile(fp):
            with open(fp, "rb") as f:
                data = f.read()
        else:
            with open(fp, "wb") as f:
                data = requests.get(url).content
                f.write(data)
        return np.frombuffer(gzip.decompress(data), dtype=np.uint8).copy()

    y_train = fetch("http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz")[0x10:].reshape((-1, 784))
    s_train = fetch("http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz")[8:]
    y_test = fetch("http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz")[0x10:].reshape((-1, 784))
    s_test = fetch("http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz")[8:]

    # taking only 1's and 7's
    y_train = y_train[(s_train == 1) | (s_train == 7)]
    s_train = s_train[(s_train == 1) | (s_train == 7)].astype(np.int8)
    y_test = y_test[(s_test == 1) | (s_test == 7)]
    s_test = s_test[(s_test == 1) | (s_test == 7)].astype(np.int8)

    # map 1's to s = 1 and 7's to s = -1
    s_train[s_train == 1] = 1
    s_train[s_train == 7] = -1
    s_test[s_test == 1] = 1
    s_test[s_test == 7] = -1

else:
    # synthetic dataset
    def make_dataset(N):
        Y = np.concatenate((np.random.multivariate_normal(np.array([-1,1]), np.identity(2), N//2), np.random.multivariate_normal(np.array([1,-1]), np.identity(2), N//2)))
        S = np.concatenate((-np.ones(N//2), np.ones(N//2)))
        return (Y, S)

    N_train = 1000
    N_test = 1000

    y_train, s_train = make_dataset(N_train)
    y_test, s_test = make_dataset(N_test)


In [3]:
model = MyClassifier(y_train.shape[1])

# # example sample selection:
# training_sample = [1,0]
# training_label = -1
# model.current_training_label = training_label
# model.sample_selection(training_sample)
# print(model.y_train)
# print(model.s_train)

In [4]:
print(y_train.shape)
print(s_train.shape)
print(y_test.shape)
print(s_test.shape)

# check for roughly equal amounts of training samples per class
print()
print((s_train[s_train == 1]).shape)
print((s_train[s_train == -1]).shape)

(1000, 2)
(1000,)
(1000, 2)
(1000,)

(500,)
(500,)


In [5]:
# train
model.train(y_train, s_train)


The optimal value is 2.878930637175079e-13


/Users/ryannemiroff/miniconda3/lib/python3.9/site-packages/cvxpy/expressions/expression.py:558: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 1 times so far.

  warnings.warn(msg, UserWarning)


In [6]:
# test
def percentage_correct(inferences, labels):
    N_test = inferences.shape[0]
    return 1/N_test * np.ones(N_test)@(labels == inferences)

inferences = model.test(y_test)
print(percentage_correct(inferences, s_test))


0.9250000000000007
